<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Обзор-данных." data-toc-modified-id="Обзор-данных.-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Обзор данных.</a></span><ul class="toc-item"><li><span><a href="#Импорт-библиотеки,-выделение-первых-строк-таблицы-и-изучение-полученной-информации-о-датафрейме." data-toc-modified-id="Импорт-библиотеки,-выделение-первых-строк-таблицы-и-изучение-полученной-информации-о-датафрейме.-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Импорт библиотеки, выделение первых строк таблицы и изучение полученной информации о датафрейме.</a></span></li></ul></li><li><span><a href="#Разделение-данных-на-выборки." data-toc-modified-id="Разделение-данных-на-выборки.-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Разделение данных на выборки.</a></span></li><li><span><a href="#Исследование-модели." data-toc-modified-id="Исследование-модели.-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Исследование модели.</a></span><ul class="toc-item"><li><span><a href="#&quot;Дерево-решений&quot;,-DecisionTreeClassifier" data-toc-modified-id="&quot;Дерево-решений&quot;,-DecisionTreeClassifier-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>"Дерево решений", DecisionTreeClassifier</a></span></li><li><span><a href="#&quot;Случайный-лес&quot;,-RandomForestClassifier" data-toc-modified-id="&quot;Случайный-лес&quot;,-RandomForestClassifier-3.2"><span class="toc-item-num">3.2&nbsp;&nbsp;</span>"Случайный лес", RandomForestClassifier</a></span></li><li><span><a href="#&quot;Линейная-регрессия&quot;,-LogisticRegression" data-toc-modified-id="&quot;Линейная-регрессия&quot;,-LogisticRegression-3.3"><span class="toc-item-num">3.3&nbsp;&nbsp;</span>"Линейная регрессия", LogisticRegression</a></span></li></ul></li><li><span><a href="#Проверка-модели-на-тестовой-выборке" data-toc-modified-id="Проверка-модели-на-тестовой-выборке-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Проверка модели на тестовой выборке</a></span></li><li><span><a href="#Проверка-модели-на-адекватность" data-toc-modified-id="Проверка-модели-на-адекватность-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Проверка модели на адекватность</a></span></li><li><span><a href="#Общий-вывод" data-toc-modified-id="Общий-вывод-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Общий вывод</a></span></li></ul></div>

# Рекомендация тарифов

Многие клиенты оператора мобильной связи «Мегалайн» пользуются архивными тарифами. «Мегалайн» хочет построить систему, способную проанализировать поведение клиентов и предложить пользователям новый тариф: «Смарт» или «Ультра». В нашем распоряжении данные о поведении клиентов, которые уже перешли на эти тарифы.

Целью данного исследования является построение модели для задачи классификации, которая выберет подходящий тариф. Задача исследования - построить модель с максимально большим значением accuracy (нужно довести долю правильных ответов по крайней мере до 0.75).

План работы включает в себя (предобработка данных уже выполнена):
- разделение исходных данных на обучающую, валидационную и тестовую выборки;
- исследование качества разных моделей с помощью изменения гиперпараметров;
- проверка качества модели на тестовой выборке;
- проверка модели на вменяемость.

Данные о поведении клиентов, которые уже перешли на эти тарифы, хранятся в файле users_behavior.csv.

Каждый объект в наборе данных — это информация о поведении одного пользователя за месяц. 
Описание колонок:
- сalls — количество звонков,
- minutes — суммарная длительность звонков в минутах,
- messages — количество sms-сообщений,
- mb_used — израсходованный интернет-трафик в Мб,
- is_ultra — каким тарифом пользовался в течение месяца («Ультра» — 1, «Смарт» — 0).

## Обзор данных.

### Импорт библиотеки, выделение первых строк таблицы и изучение полученной информации о датафрейме.

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.dummy import DummyClassifier
import warnings
warnings.filterwarnings("ignore")

In [2]:
def my_func(x):
    print('Исходный датафрейм')
    display(x.head())
    print('Общая информация о датафрейме')
    display(x.info())
    print('Количество пустых значений в датафрейме')
    display(x.isna().sum())
    print('Количество явных дубликатов в датафрейме')
    display(x.duplicated().sum())
    print('Описательная статистика')
    display(x.describe())
    
try:
    data = pd.read_csv('/datasets/users_behavior.csv')
except:
    data = pd.read_csv('C:/Users/daria/Датасеты для проектов/06 Введение в машинное обучение - рекомендация тарифов/users_behavior.csv')
my_func(data)

Исходный датафрейм


,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0


Общая информация о датафрейме
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     3214 non-null   float64
 1   minutes   3214 non-null   float64
 2   messages  3214 non-null   float64
 3   mb_used   3214 non-null   float64
 4   is_ultra  3214 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


None

Количество пустых значений в датафрейме


calls       0
minutes     0
messages    0
mb_used     0
is_ultra    0
dtype: int64

Количество явных дубликатов в датафрейме


0

Описательная статистика


,calls,minutes,messages,mb_used,is_ultra
count,3214.000000,3214.000000,3214.000000,3214.000000,3214.000000
mean,63.038892,438.208787,38.281269,17207.673836,0.306472
std,33.236368,234.569872,36.148326,7570.968246,0.461100
min,0.000000,0.000000,0.000000,0.000000,0.000000
25%,40.000000,274.575000,9.000000,12491.902500,0.000000
50%,62.000000,430.600000,30.000000,16943.235000,0.000000
75%,82.000000,571.927500,57.000000,21424.700000,1.000000
max,244.000000,1632.060000,224.000000,49745.730000,1.000000


**Вывод:**

При первичном анализе было выявлено, что пустых значений в столбцах нет, как и явных дубликатов. Явных аномалий также найдено не было.

Целевой признак в данном исследовании - данные в столбце 'is_ultra'.

## Разделение данных на выборки.

В данном исследовании спрятанной тестовой выборки нет, значит, данные следует разбить на три части: обучающую, валидационную и тестовую в соотношении 3:1:1. На обучающую выборку отводится 60% исходных данных, на валидационную и тестовую выборки по 20%. Разделим сначала исходные данные на обучающую выборку и оставшуюся часть выборки (40%), после этого остаток разделим пополам, чтобы получилось равное количество данных в валидационной и тестовой выборках.

In [3]:
train_data, valid_data, test_data = np.split(data.sample(frac=1, random_state=821), \
                              [int(.6*len(data)), int(.8*len(data))])
print('Размер обучающей выборки', train_data.shape[0])
print('Размер валидационной выборки', valid_data.shape[0])
print('Размер тестовой выборки', test_data.shape[0])

Размер обучающей выборки 1928
Размер валидационной выборки 643
Размер тестовой выборки 643


Теперь определим целевой признак и остальные признаки по каждой выборке.

In [4]:
train_data_features = train_data.drop(['is_ultra'], axis=1)
train_data_target = train_data['is_ultra']

In [5]:
valid_data_features = valid_data.drop(['is_ultra'], axis=1)
valid_data_target = valid_data['is_ultra']

In [6]:
test_data_features = test_data.drop(['is_ultra'], axis=1)
test_data_target = test_data['is_ultra']

**Вывод:**

В этом блоке исследования разделили исходные данные на три части: обучающую, валидационную и тестовую выборки в соотношении 3:1:1. По каждому из выборок определили целевой признак - данные в столбце 'is_ultra', а также остальные признаки.

## Исследование модели.

### "Дерево решений", DecisionTreeClassifier

Определим точность модели **"Дерево решений", DecisionTreeClassifier** и найдем лучшие значения гиперпараметров 'max_depth', 'min_samples_split', 'min_samples_leaf', 'criterion'.

In [7]:
best_parameters = []
best_result = 0
for depth in range(1, 10):
    for samples_split in range(2, 10):
            for samples_leaf in range(1, 10):
                for criterion in ['gini', 'entropy']:
                    model = DecisionTreeClassifier(
                        random_state=12345, 
                        max_depth=depth,
                        min_samples_split=samples_split,
                        min_samples_leaf=samples_leaf,
                        criterion=criterion
                    )
                    model.fit(train_data_features, train_data_target)
                    predictions = model.predict(valid_data_features)
                    result = accuracy_score(valid_data_target, predictions)
                    if result > best_result:
                        best_parameters = [depth, samples_split, samples_leaf, criterion]
                        best_result = result        

In [8]:
print('Лучшая глубина дерева:', best_parameters[0])
print('Лучшее количество выборок:', best_parameters[1])
print('Лучшее количество образцов:', best_parameters[2])
print('Лучший критерий:', best_parameters[3])
print('Accuracy модели "Дерево решений" на валидационной выборке:', best_result)

Лучшая глубина дерева: 7
Лучшее количество выборок: 2
Лучшее количество образцов: 4
Лучший критерий: entropy
Accuracy модели "Дерево решений" на валидационной выборке: 0.8009331259720062


Accuracy модели "Дерево решений" равно 80%.

### "Случайный лес", RandomForestClassifier

Определим точность модели **"Случайный лес", RandomForestClassifier**, и найдем лучшие значения гиперпараметров 'n_estimators', 'max_depth', 'min_samples_split', 'min_samples_leaf', 'criterion'.

In [9]:
best_parameters = []
best_result = 0
for est in range(1, 10):
    for depth in range(1, 10):
        for samples_split in range(2, 10):
                for samples_leaf in range(1, 10):
                    for criterion in ['gini', 'entropy']:
                        model_2 = RandomForestClassifier(
                            random_state=12345,
                            n_estimators=est,
                            max_depth=depth,
                            min_samples_split=samples_split,
                            min_samples_leaf=samples_leaf,
                            criterion=criterion
                        )
                        model_2.fit(train_data_features, train_data_target)
                        predictions = model_2.predict(valid_data_features)
                        result = accuracy_score(valid_data_target, predictions)
                        if result > best_result:
                            best_parameters = [est, depth, samples_split, samples_leaf, criterion]
                            best_result = result        

In [10]:
print('Лучшее количество деревьев:', best_parameters[0])
print('Лучшая глубина дерева:', best_parameters[1])
print('Лучшее количество выборок:', best_parameters[2])
print('Лучшее количество образцов:', best_parameters[3])
print('Лучший критерий:', best_parameters[4])
print('Accuracy модели "Случайный лес" на валидационной выборке:', best_result)

Лучшее количество деревьев: 4
Лучшая глубина дерева: 8
Лучшее количество выборок: 2
Лучшее количество образцов: 7
Лучший критерий: entropy
Accuracy модели "Случайный лес" на валидационной выборке: 0.8180404354587869


Точность модели "Случайный лес" улучшена на 1% (по сравнению с моделью "Дерево решений", и составляет 81%.

### "Линейная регрессия", LogisticRegression

Определим точность модели **линейной регрессии, LogisticRegression**:

In [11]:
best_model_3 = None
best_result = 0
best_solver = ''
solvers = ['liblinear', 'newton-cg', 'lbfgs', 'sag', 'saga']

for solver in solvers:
    model_3 = LogisticRegression(random_state=12345, solver=solver)
    model_3.fit(train_data_features, train_data_target)
    predictions = model_3.predict(valid_data_features)
    result = accuracy_score(valid_data_target, predictions)
    if result > best_result:
        best_result = result
        best_model_3 = model_3
        best_solver = solver                
        
print('Лучший алгоритм, используемый для оптимизации:', best_solver)        
print('Accuracy модели "Логистическая регрессия" на валидационной выборке:', best_result)

Лучший алгоритм, используемый для оптимизации: newton-cg
Accuracy модели "Логистическая регрессия" на валидационной выборке: 0.7480559875583204


**Вывод:**

Из исследования моделей выше можно сделать вывод, что модель "случайный лес" является моделью с лучшим показателем точности на валидационной выборке - 81%. Лучшие гиперпараметры следующие:

- Лучшее количество деревьев: 4
- Лучшая глубина дерева: 8
- Лучшее количество выборок: 2
- Лучшее количество образцов: 7
- Лучший критерий: entropy

Остальные модели, которые были исследованы также в этом этапе: "дерево решений" и "логистическая регрессия".

Лучший показатель точности модели "Дерево решений" на валидационной выборке составил 80%. При этом были определены лучшие гиперпараметры: 
- Лучшая глубина дерева: 7
- Лучшее количество выборок: 2
- Лучшее количество образцов: 4
- Лучший критерий: entropy

Что касается модели "линейная регрессия", то лучший алгоритм, используемый для оптимизации, это - 'newton-cg'. В то время как точность модели составила 74%.

## Проверка модели на тестовой выборке

Выполним проверку модели на тестовой выборке, используя модель с лучшим показателем accuracy:

In [12]:
final_model = RandomForestClassifier(
    random_state=12345,
    n_estimators=8,
    max_depth=6,
    min_samples_split=9,
    min_samples_leaf=4,
    criterion='gini'
)
final_model.fit(train_data_features, train_data_target)
predictions = final_model.predict(test_data_features)
result = accuracy_score(test_data_target, predictions)
print('Accuracy модели "Случайный лес" на тестовой выборке:', result)

Accuracy модели "Случайный лес" на тестовой выборке: 0.8055987558320373


**Вывод:**

Проверили модель "Случайный лес", как модель с лучшим показателем точности - 81%, и можем сделать вывод о небольшой переобученности модели - значение показателя точности на тестовой выборке хуже - 80%. Задачей исследования являлась необходимость построения модели с максимально большим значением accuracy - нам удалось получить долю правильных ответов выше 75%.

## Проверка модели на адекватность

Проверим модель на адекватность с использованием классификатора DummyClassifier:

In [13]:
dummy_clf = DummyClassifier(strategy='most_frequent', random_state=12345)
dummy_clf.fit(train_data_features, train_data_target)
predictions = dummy_clf.predict(valid_data_features)
result = accuracy_score(valid_data_target, predictions)

print('Accuracy dummy-модели: ', result)

Accuracy dummy-модели:  0.6936236391912908


**Вывод:**

Проверка выше доказала, что точность модели DummyClassifier ниже, чем у модели случайного леса - модель адекватна и эффективна.

## Общий вывод

При первичном анализе было выявлено, что пустых значений в столбцах нет, как и явных дубликатов. Явных аномалий также найдено не было. Целевым признаком определили данные в столбце 'is_ultra'.

Далее были разделены исходные данные на три части: обучающую, валидационную и тестовую выборки в соотношении 3:1:1.

Исследование моделей показало, что модель "случайный лес" является моделью с лучшим показателем точности на валидационной выборке - 81%. Лучшие гиперпараметры следующие:

- Лучшее количество деревьев: 4
- Лучшая глубина дерева: 8
- Лучшее количество выборок: 2
- Лучшее количество образцов: 7
- Лучший критерий: entropy

Остальные модели, которые были исследованы также в этом этапе: "дерево решений" и "логистическая регрессия".

Лучший показатель точности модели "Дерево решений" на валидационной выборке составил 80%. При этом были определены лучшие гиперпараметры: 
- Лучшая глубина дерева: 7
- Лучшее количество выборок: 2
- Лучшее количество образцов: 4
- Лучший критерий: entropy

Что касается модели "линейная регрессия", то лучший алгоритм, используемый для оптимизации, это - 'newton-cg'. В то время как точность модели составила 74%.

Проверили модель "Случайный лес", как модель с лучшим показателем точности - 81%, и можем сделать вывод о небольшой переобученности модели - значение показателя точности на тестовой выборке хуже - 80%. Задачей исследования являлась необходимость построения модели с максимально большим значением accuracy - нам удалось получить долю правильных ответов выше 75%.

Проверка модели на адекватность доказала, что точность модели DummyClassifier ниже, чем у модели случайного леса - модель адекватна и эффективна.